In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append('../')
from ck_analysis import get_repo_id

repo_names = ["pay-as-you-go-api", "paygo-telepase-api", "prepaid-api", "sp-utilities-provider-api", "paygo-taggy-api"]

for repo_name in repo_names:
    df = pd.read_csv(f'../output/deploys/raw/{repo_name}_deploys.csv')
    df.columns = df.columns.str.strip()
    df["PERIOD_DATE"] = pd.to_datetime(df["period_date (Año y mes)"], format='%b %Y', errors='coerce', dayfirst=True)
    df["DEPLOYS"] = df["Deploys"]
    df["ROLLBACKED_DEPLOYS"] = df["Rollbacked Deploys"]
    df = df[['PERIOD_DATE', 'DEPLOYS', 'ROLLBACKED_DEPLOYS']]
    df.to_csv(f'../output/deploys/{get_repo_id(repo_name)}_deploys.csv', index=False)



In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append('../')
from ck_analysis import get_repo_id

df = pd.read_csv('../output/static_analisys/raw/static_analisys_issues_master.csv')
df.columns = df.columns.str.strip()
df["CREATE_DTTM"] = pd.to_datetime(df["CREATE_DTTM"])
df["REPO_NAME"] = df["APPLICATION_NAME"].apply(get_repo_id)

severity_weights = {'low': 1, 'medium': 3, 'high': 5}
df['TOTAL_ISSUES_WEIGHTED'] = df['ISSUE_SEVERITY'].map(severity_weights) * df['TOTAL_ISSUES_QTY']

df = df.groupby(["CREATE_DTTM", "REPO_NAME", "PROVIDER"]).agg({"TOTAL_ISSUES_WEIGHTED": "sum", "TOTAL_ISSUES_QTY": "sum"}).reset_index()
df.to_csv('../output/static_analisys/static_analisys_master.csv', index=False)

In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('../output/static_analisys/static_analisys_issues.csv')

# obtener los distintos valores de la combinacion de coliumnas PROVIDER y ISSUE_SEVERITY
df = df[["ISSUE_SEVERITY", "PROVIDER"]].drop_duplicates()
df.sort_values(by=["PROVIDER"], ascending=False)

In [ ]:
import pandas as pd

def weighted_average(group, value_col, weight_col):
    return (group[value_col] * group[weight_col]).sum() / group[weight_col].sum()


df = pd.read_csv('../datasets/apps_ck_monthly.csv')
# Strip any leading or trailing spaces from column names
df.columns = df.columns.str.strip()

# Remove any rows where the type is enum
df = df[df['type'] != 'enum']
# Remove any rows where the class name contains ".dto." 
df = df[~df['class'].str.contains('.dto.')]

df = df[['tag_date', 'tag_name', 'class', 'wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'loc']]
df.columns = ['TagDate', 'TagName', 'ClassName', 'WMC', 'DIT', 'NOC', 'CBO', 'RFC', 'LCOM', 'LOC']
df['TagDate'] = pd.to_datetime(df['TagDate'])

grouped_metrics = df.groupby('TagDate')[['WMC', 'DIT', 'NOC', 'CBO', 'RFC', 'LCOM']]

median_by_date = grouped_metrics.median()
mean_by_date = grouped_metrics.mean()

# Crear un diccionario de funciones de agregación
agg_funcs = {metric: lambda x, col=metric: weighted_average(df.loc[x.index], col, 'LOC') 
            for metric in ['WMC', 'DIT', 'NOC', 'CBO', 'RFC', 'LCOM']}

weighted_avg_by_date = grouped_metrics.agg(agg_funcs)

weighted_avg_by_date
